In [1]:
# Combine all the coefficients into a dataframe 
coefficients = list() 
 
coeff_labels = ['lr', 'l1', 'l2']
coeff_models = [lr, lr_l1, lr_l2]

for lab, mod in zip(coeff_labels, coeff_models): 
    coeffs = mod.coef_
    coeff_label = pd.MultiIndex(levels = [[lab], [0,1,2,3,4,5]], 
                                labels = [[0,0,0,0,0,0], [0,1,2,3,4,5]])
    coefficients.append(pd.DataFrame(coeffs.T, columns = coeff_label))
    # .T transposes index and columns 
    # coefficients is a list of 3 (lr, lr_l1, lr_l2) matrices (561x6)

# concatenate matrices by column     
coefficients = pd.concat(coefficients, axis = 1)

NameError: name 'lr' is not defined

In [ ]:
# Plot 6 plots of each multi-class coefficient 
fig, axList = plt.subplots(nrows = 3, ncols = 2)
axList = axList.flatten()
fig.set_size_inches(10,10)

# iterate through each classification 0,1,2,3,4,5
for ax in enumerate(axList): 
    loc = ax[0] # loc is 0,1,2,3,4,5 for each plot/classification
    ax = ax[1]
    # print(loc, ax)
    
    # .xs() returns column (axis = 1) of coefficients for each class (loc)
    data = coefficients.xs(loc, level = 1, axis = 1) # level 1 is labels (lr,..)
    # print (data.head(10)) # data is all the coefficients for each class
    data.plot(marker = 'o', ls = '', ms = 2.0, ax = ax, legend = False)
    
    if ax is axList[0]: # if 1st plot, plot legend
        ax.legend(loc = 4) # location of legend (loc=4 is 'lower right')
    
    ax.set(title = 'Coefficient Set' + str(loc))

    plt.tight_layout()

In [ ]:
# Predict the class and the probability for each 

y_pred = list()
y_prob = list() 

coeff_labels = ['lr', 'l1', 'l2']
coeff_models = [lr, lr_l1, lr_l2]

for lab, mod in zip(coeff_labels, coeff_models): 
    y_pred.append(pd.Series(mod.predict(X_test), 
                            name = lab))
    y_prob.append(pd.Series(mod.predict_proba(X_test)
                            .max(axis = 1),  # prob of highest class
                            name = lab))
    
y_pred = pd.concat(y_pred, axis = 1)
y_prob = pd.concat(y_prob, axis = 1)


In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score 
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize

metrics = list() 
cm = dict() 

for lab in coeff_labels: 
    
    # precision, recall, f-score from the multi-class support function 
    precision, recall, fscore, __ = score(y_test, y_pred[lab], 
                                          average = 'weighted')
    
    # usual way to calculate accuracy 
    accuracy = accuracy_score(y_test, y_pred[lab])
    
    # ROC-AUC scores can be calculated by binarizing the data 
    auc = roc_auc_score(label_binarize(y_test, classes = [0,1,2,3,4,5]), 
                        label_binarize(y_pred[lab], classes = [0,1,2,3,4,5]), 
                        average = 'weighted')
    
    # confusion matrix 
    cm[lab] = confusion_matrix(y_test, y_pred[lab])
    
    metrics.append(pd.Series({'precision': precision, 
                              'recall': recall, 
                              'fscore': fscore, 
                              'accuracy': accuracy, 
                              'auc': auc}, name = lab))
    
metrics = pd.concat(metrics, axis = 1)
metrics 

In [ ]:
# Plot confusion matrix 
fig, axList = plt.subplots(nrows = 2, ncols = 2)
axList = axList.flatten()
fig.set_size_inches(12, 10)

axList[-1].axis('off') # takes away 4th plot 

for ax, lab in zip(axList[:-1], coeff_labels): 
    sns.heatmap(cm[lab], 
                ax = ax, 
                annot = True, # annotate each cell w/ number
                fmt = 'd'); # annotate with integer format (as opposed to 5e83)
    ax.set(title = lab); 
    
plt.tight_layout()